# Demo KartAI

## Import pakker

In [2]:
import os
import fiona
import pyarrow.parquet as pq
import geopandas as gpd
import pandas as pd
import time as time_module
import glob
import datetime
import json
import re
import builtins
from geopandas import GeoDataFrame
from shapely import wkt
from shapely.geometry import box
from IPython.display import display
from typing import Union

## Konstante variabler

In [3]:
ABSOLUTE_PATH = os.getcwd()
ROT_FOLDER = "Data"
ROT_PATH = os.path.join(ABSOLUTE_PATH, ROT_FOLDER)
SUPPORTED_FORMATS = ['.parquet', '.geojson', '.json', '.shp', '.gpkg', '.csv', '.gml']
FOLDER = {
    'raw': os.path.join(ROT_PATH, 'raw'), 
    'processed': os.path.join(ROT_PATH, 'processed'), 
    'archive': os.path.join(ROT_PATH, 'archive'),
    'logs': os.path.join(ROT_PATH, 'logs')
}

LOG_FILE = os.path.join(FOLDER['logs'], 'conversion_log.json')

## Diverse Sjekker

In [4]:
def valid_conversion(filepath, filename, filetype):
    """Sjekker om en fil er gyldig for konvertering."""
    
    # Ikke konverter loggfiler
    if filename == "conversion_log" or filepath.endswith("conversion_log.json"):
        return False

    # Returner False hvis formatet ikke er støttet
    if filetype not in SUPPORTED_FORMATS:
        return False

    # Håndtering av Parquet-filer
    if (filetype == ".parquet" and filename.endswith("_geo")) and (not os.path.isfile(filepath)):
        return False

    return True

In [5]:
def check_file_convert(filepath, filename, convert_files, convert_logs):
    """Sjekker om en fil trenger konvertering basert på konverteringshistorikk"""
    # Sjekk loggfilen
    if filename in convert_logs:
        # Sjekk om den konverterte filen faktisk eksisterer
        target_path = convert_logs[filename].get('målsti', None)
        if target_path and os.path.exists(target_path):
            # Filen finnes - sjekk om den har blitt endret siden sist konvertering
            sist_endret_tid = os.path.getmtime(filepath)
            sist_konvertert_tid = convert_logs[filename].get('tidspunkt', 0)
            if sist_endret_tid <= sist_konvertert_tid:
                return False
        else:
            # Filen i loggfilen eksisterer ikke - den bør konverteres på nytt
            print(f"Konvertert fil {target_path} finnes ikke, konverterer på nytt...")
            return True

    # Sjekk om filen allerede finnes i processed-mappen
    return filename not in convert_files

In [6]:
def check_col_exists(df, col):
    return col in df.columns


## Get-funskjoner

In [7]:
def get_convert_files():
    """Henter ut alle allerede konverterte filer"""

    # Return an empty set if the folder does not exist
    if not os.path.exists(FOLDER['processed']):
        return None

    convert_files = set()
    # Iterate through files in the directory
    for root, _, filer in os.walk(FOLDER['processed']):
        for fil in filer:
            if fil.endswith('.parquet'):
                base_navn = os.path.splitext(fil)[0]
                convert_files.add(base_navn)

    return convert_files

In [8]:
def get_exists_geoparquet():
    """Finner eksisterende GeoParquet-filer"""
    geoparquet_files = []

    for root, _, files in os.walk(FOLDER['processed']):
        for file in files:
            if file.endswith('.parquet'):
                filepath = os.path.join(root, file)
                if filepath not in files:
                    geoparquet_files.append(filepath)
    print(f"Antall geoparquet filer: {geoparquet_files}")
    return geoparquet_files

In [9]:
def get_new_files():
    """Finner filer som ikke har blitt konvertert ennå"""
    convert_files = get_convert_files()
    #convert_logs = read_log()

    nye_filer = []

    for root, _, files in os.walk(FOLDER['raw']):
        for file in files:
            filepath = os.path.join(root, file)
            filetype = os.path.splitext(file)[1].lower()
            filename = os.path.splitext(file)[0]

            # Sjekk om filen er gyldig for konvertering
            if not valid_conversion(filepath, filename, filetype):
                print(f"fil {filename} er ikke gyldig for konverting")
                continue

            # Sjekk om filen trenger konvertering
            #if check_file_convert(filepath, filename, convert_files):
            nye_filer.append(filepath)
    print(f"Hente {len(nye_filer)} ny filer")
    return nye_filer

In [10]:
def get_geoparquet_file(filepath) -> GeoDataFrame:
    # Read the Parquet file
    geo_df = gpd.read_parquet(filepath)
        # Ensure it's a GeoDataFrame
    if not isinstance(geo_df, gpd.GeoDataFrame):
        geo_df = gpd.GeoDataFrame(geo_df, geometry="geometry")
    
    return geo_df

### Konverting til parquet

#### sjekk om det er Parquet

In [11]:
def converter_parquet(filepath):
    """Konverterer parquet-fil til GeoDataFrame"""
    if os.path.exists(filepath):
        df = pd.read_parquet(filepath)
    
        # Try latitude/longitude conversion first
        gdf = convert_latlon_to_gdf(df)
        if gdf is not None:
            return gdf
    
        # Try detecting and converting WKT geometry
        gdf = detect_wkt_geometry(df)
        if gdf is not None:
            return gdf

    print("No valid geometry columns found.")
    return None
    

In [12]:
def convert_latlon_to_gdf(df):
    """Convert a DataFrame with latitude/longitude columns to a GeoDataFrame."""
    if {'longitude', 'latitude'}.issubset(df.columns):
            df['geometry'] = gpd.points_from_xy(df['longitude'], df['latitude'])
            return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
    return None

In [13]:
def detect_wkt_geometry(df):
    """Detect a potential geometry column in WKT format and convert it to a GeoDataFrame."""
    pattern = re.compile(r'geom|coord|point|polygon|linestring|wkt', re.IGNORECASE)
    geo_col = [col for col in df.columns if pattern.search(col)]

    for col in geo_col:
        if df[col].dtype == 'object':
            # Prøv å konvertere fra WKT-format
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
    return None

#### Sjekk om det er CSV

In [14]:
def converter_csv(filsti):
    """Konverterer CSV-fil til GeoDataFrame"""
    df = pd.read_csv(filsti)

    # Sjekk for vanlige lat/long kolonnenavn
    lat_kolonner = ['latitude', 'lat', 'y', 'breddegrad']
    lon_kolonner = ['longitude', 'long', 'lon', 'x', 'lengdegrad']

    lat_col = next((col for col in lat_kolonner if col in df.columns), None)
    lon_col = next((col for col in lon_kolonner if col in df.columns), None)

    if lat_col and lon_col:
        return gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
            crs="EPSG:4326"
        )

    return None

#### Sjekk om det er gml

In [15]:
def convert_gml(path):
    layers = fiona.listlayers(path)
    if layers:
        print(f"GML-fil har {len(layers)} lag: {', '.join(layers)}")
        # Les det første laget, som er standard
        return gpd.read_file(path, layer=layers[0])
    else:
        return gpd.read_file(path)

#### Konvertere filer til geoparquet

In [16]:
def convert_filetype(source_filepath: str) -> Union[gpd.GeoDataFrame, None]:
    """Konverterer fil til GeoDataFrame basert på filendelse"""
    
    filetype = os.path.splitext(source_filepath)[1].lower()
    if filetype == '.parquet':
        return converter_parquet(source_filepath)

    elif filetype in ['.geojson', '.json', '.shp', '.gpkg']:
        return gpd.read_file(source_filepath)

    elif filetype == '.gml':
        return convert_gml(source_filepath)
    
    elif filetype == '.csv':
        return converter_csv(source_filepath)

    print(f"Klarte ikke å finne match konverting til filendelse {filetype}")
    return None

In [17]:
def converter_to_geoparquet(source_filepath: str):
    """
    Konverterer en enkelt fil til GeoParquet-format.
    Hvis ingen målmappe er spesifisert, brukes FOLDER['processed'].
    Returnerer den nye filbanen dersom konvertering lykkes, ellers False.
    """
    # Bygg målfilstien basert på kildefilens navn med '_geo.parquet' suffix
    filename = os.path.basename(source_filepath)
    filename_without_ext = os.path.splitext(filename)[0]
    target_file_path = os.path.join(FOLDER['processed'], f"{filename_without_ext}_geo.parquet")
    
    # Konverter filen ved hjelp av en antatt funksjon convert_filetype som returnerer en GeoDataFrame
    gdf = convert_filetype(source_filepath)

    if gdf is not None:
        gdf.to_parquet(target_file_path)
        return target_file_path
    return None


### Logging

In [18]:
def read_log() -> dict:
    """
    Leser inn eksisterende konverteringslogg.
    Hvis loggfilen ikke finnes, returneres en tom dictionary.
    """
    """Leser inn eksisterende konverteringslogg"""
    if not os.path.exists(LOG_FILE):
        return {}
    with builtins.open(LOG_FILE, 'r', encoding='utf-8') as f:
        return json.load(f)


In [19]:
def generate_logs(file: str, target_file_path: str) -> dict:
    """
    Lager et nytt logginnslag for en konvertert fil.
    """
    base_name = os.path.splitext(os.path.basename(file))[0]
    today = datetime.datetime.now()
    return {
        base_name: {
            'kildesti': file,
            'målsti': target_file_path,
            'tidspunkt': today.timestamp(),
            'dato': today.strftime('%Y-%m-%d %H:%M:%S')
        }
    }

In [20]:
def save_log(logdata: dict) -> None:
    """
    Lagrer konverteringslogg til fil.
    """
    with open(LOG_FILE, 'w', encoding='utf-8') as f:
        json.dump(logdata, f, indent=2)

In [21]:
def update_logs(file: str, target_file_path: str) -> None:
    """
    Oppdaterer konverteringsloggen med ny filinfo.
    """
    convert_logger = read_log()
    new_log = generate_logs(file, target_file_path)
    convert_logger.update(new_log)
    save_log(convert_logger)

## Setup

In [22]:
def setup():
    # Sjekk om mappen finnes, hvis ikke - lag den
    if not os.path.exists(ROT_FOLDER):
        os.makedirs(ROT_FOLDER, exist_ok=True)
        print(f"Opprettet rotmappe: {ROT_FOLDER}")
    else:
        print(f"Mappen {ROT_FOLDER} finnes allerede")
        
    # Opprett undermappe-struktur
    for folderName in FOLDER:
        if not os.path.exists(FOLDER[folderName]):
            print(f"Creating folder in {ROT_FOLDER}")
            os.makedirs(FOLDER[folderName], exist_ok=True)
            print(f"Opprettet mappe: {FOLDER[folderName]}")
            
    

In [23]:
def skriv_oppsummering(resultater):
    """Skriver oppsummering av konverteringsprosessen"""
    print("\n")
    print("OPPSUMMERING")
    print(f"Katalog: {ROT_PATH}")
    print(f"Mappestruktur: Raw={FOLDER['raw']}, Processed={FOLDER['processed']}")
    
    print(f"Totalt antall geografiske filer funnet: {len(resultater["alle_filer"])}")
    print(f"Antall parquet-filer som allerede var GeoParquet: {len(resultater["allerede_geoparquet"])}")
    print(f"Antall filer konvertert til GeoParquet: {len(resultater["konvertert"])}")
    print(f"Antall filer som ikke kunne konverteres: {len(resultater["feilet"])}")

    if resultater["konvertert"]:
        print("\nKonverterte filer:")
        for fil in resultater["konvertert"]:
            print(f"- {fil}")

    if resultater["feilet"]:
        print("\nFiler som ikke kunne konverteres:")
        for fil in resultater["feilet"]:
            print(f"- {fil}")

In [24]:
def behandle_alle_filer_i_mappe():
    """Behandler alle geografiske filer i angitt mappe og konverterer til GeoParquet."""
    # Validering og mappestruktur (gjør dette først!)
    if not os.path.exists(ROT_PATH):
        print(f"Vennligst oppgi en gyldig mappe som du har tilgang til.")
        print(f"For eksempel: './data' (relativ sti) eller '~/data' (i hjemmekatalogen)")
        return None

    # Bruk processed-sti som målmappe i stedet for prosessert_mappesti fra oppsett_mappestier
    if not os.path.exists(FOLDER['processed']):
        print(f"FEIL: Kunne ikke opprette målmappen '{FOLDER['processed']}': {e}")
        return None

    # Finn nye filer som trenger konvertering
    new_files = get_new_files()
    
    # Konverter filene og samle resultatene
    konvertert = []
    feilet = []
    for filepath in new_files:
        target_filepath = converter_to_geoparquet(filepath)
        if target_filepath is not None:
            konvertert.append(filepath)
            update_logs(FOLDER['logs'], filepath)
        else:
            feilet.append(filepath)

    # Sammenstill resultater
    resultater = {
        "alle_filer": new_files,
        "konvertert": konvertert,
        "feilet": feilet,
        "allerede_geoparquet": get_exists_geoparquet()
    }
    skriv_oppsummering(resultater)

    return resultater

In [25]:
# Sjekk at mappen eksisterer før vi fortsetter
setup()

Mappen Data finnes allerede


In [26]:
# Kjører konverteringsprosessen
resultater = behandle_alle_filer_i_mappe()

Hente 366 ny filer
Antall geoparquet filer: ['C:\\Users\\ANDRI\\Documents\\PythonProjects\\kaidata_geolake\\Data\\processed\\hais_2024-01-01.snappy_geo.parquet', 'C:\\Users\\ANDRI\\Documents\\PythonProjects\\kaidata_geolake\\Data\\processed\\hais_2024-01-02.snappy_geo.parquet', 'C:\\Users\\ANDRI\\Documents\\PythonProjects\\kaidata_geolake\\Data\\processed\\hais_2024-01-03.snappy_geo.parquet', 'C:\\Users\\ANDRI\\Documents\\PythonProjects\\kaidata_geolake\\Data\\processed\\hais_2024-01-04.snappy_geo.parquet', 'C:\\Users\\ANDRI\\Documents\\PythonProjects\\kaidata_geolake\\Data\\processed\\hais_2024-01-05.snappy_geo.parquet', 'C:\\Users\\ANDRI\\Documents\\PythonProjects\\kaidata_geolake\\Data\\processed\\hais_2024-01-06.snappy_geo.parquet', 'C:\\Users\\ANDRI\\Documents\\PythonProjects\\kaidata_geolake\\Data\\processed\\hais_2024-01-07.snappy_geo.parquet', 'C:\\Users\\ANDRI\\Documents\\PythonProjects\\kaidata_geolake\\Data\\processed\\hais_2024-01-08.snappy_geo.parquet', 'C:\\Users\\ANDRI\\

## Hovedfunskjoner

### Filtrering

In [27]:
def filtering_time(gdf: GeoDataFrame, time: str) -> None:
    # Filtrering basert på attributt/kolonne
    if 'date_time_utc' in gdf.columns:
        filtered_by_date = gdf[gdf['date_time_utc'] >= time]
        print(f"Antall rader etter datofiltrering: {len(filtered_by_date)}")
        display(filtered_by_date.head())

In [28]:
def filtering_geomtry(gdf: GeoDataFrame, area: box) -> None:
    # Filtrering basert på geometri (f.eks. et område)
    
    within_area = gdf[gdf.geometry.intersects(area)]
    print(f"Antall rader innenfor det definerte området: {len(within_area)}")
    display(within_area.head())

In [29]:
def filtering_multiply_criteria(gdf: GeoDataFrame, area: box) -> None:
    # Kombinert filtrering med flere kriterier
    if 'attributt' in gdf.columns:
        kombinert_filter = gdf[(gdf.geometry.intersects(area)) & (gdf['attributt'] > 10)]
        print(f"Antall rader etter kombinert filtrering: {len(kombinert_filter)}")
        display(kombinert_filter.head())

In [30]:
def filtering_col(df, cols):
    valid_cols = []
    for col in cols:
        if check_col_exists(df, col):
            valid_cols.append(col)
        else:
            print(f"The column {col} don't exists in the dataframe")
    new_df = df[valid_cols].copy()
    return new_df

In [ ]:
def filtering_rad(df, search_word):
    # Convert search word to lowercase for case-insensitive search
    search_word = str(search_word).lower()
    
    # Create a boolean mask where any column contains search_word
    matches = df.apply(lambda row: row.astype(str).str.lower().str.contains(search_word, na=False), axis=1).any(axis=1)

    # Check if there are any matches
    if not matches.any():
        print(f"'{search_word}' is not in the dataframe")
        return df.iloc[0:0]  # Return an empty DataFrame instead of None
    
    return df[matches]  # Return only matching rows

In [31]:
# Les geoparquet-filen
filepath = f"{FOLDER['raw']}/hais_2024-01-01.snappy.parquet"
gdf = get_geoparquet_file(filepath)
filtering_time(gdf, '2025-01-01 16:00:00')
filtering_geomtry(gdf, box(5.0, 60.0, 11.0, 60.0))
filtering_multiply_criteria(gdf, box(5.0, 60.0, 11.0, 60.0))

Antall rader etter datofiltrering: 0


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,callsign,ship_name,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry


Antall rader innenfor det definerte området: 0


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,callsign,ship_name,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry


### Partisjonerte

In [33]:
# 1. Opprett en mappe for partisjonerte data
output_mappe = "Data/partitioned_hour"
os.makedirs(output_mappe, exist_ok=True)

# 2. Lag en kopi for å unngå SettingWithCopyWarning
within_area = gdf[gdf.geometry.intersects(box(5.0, 60.0, 11.0, 60.0))]
data_for_partisjonering = within_area.copy()

# 3. Bruk .loc for å legge til time-kolonnen
data_for_partisjonering.loc[:, 'time'] = data_for_partisjonering['date_time_utc'].dt.hour

# 4. Partisjonering basert på time og ship_type
for time, gruppe_time in data_for_partisjonering.groupby('time'):
    time_mappe = os.path.join(output_mappe, f"time={time:02d}")
    os.makedirs(time_mappe, exist_ok=True)
    
    for ship_type, gruppe_final in gruppe_time.groupby('ship_type'):
        skip_mappe = os.path.join(time_mappe, f"ship_type={ship_type}")
        os.makedirs(skip_mappe, exist_ok=True)
        
        # Bruk en kopi av dataene
        data_å_lagre = gruppe_final.copy()
        
        # Fjern partisjoneringskolonnen før lagring
        fil_sti = os.path.join(skip_mappe, f"data.parquet")
        data_å_lagre.drop('time', axis=1).to_parquet(fil_sti)
        
        print(f"Skrevet {len(gruppe_final)} rader til {fil_sti}")

In [34]:
partisjon_mappe = "Data/partitioned_hour"

# Hente data for en spesifikk måned
ønsket_måned = "2025-01"
måned_sti = os.path.join(partisjon_mappe, f"år_måned={ønsket_måned}")

if os.path.exists(måned_sti):
    print(f"Leser data for {ønsket_måned}...")
    # Finn alle geoparquet-filer i denne månedens mappe (inkludert undermapper)
    filer = glob.glob(os.path.join(måned_sti, "**/*.parquet"), recursive=True)
    # Les og kombiner alle filene
    dataframes = []
    for fil in filer:
        gdf = gpd.read_parquet(fil)
        dataframes.append(gdf)
    
    if dataframes:
        månedsdata = pd.concat(dataframes)
        print(f"Hentet {len(månedsdata)} rader for {ønsket_måned}")
        display(månedsdata.head())
    else:
        print(f"Ingen data funnet for {ønsket_måned}")
else:
    print(f"Ingen mappe funnet for {ønsket_måned}")

# Hente data for en spesifikk skipstype
ønsket_skipstype = 30
skipstype_stier = glob.glob(os.path.join(partisjon_mappe, f"**/ship_type={ønsket_skipstype}/*.parquet"), recursive=True)

if skipstype_stier:
    print(f"\nLeser data for skipstype {ønsket_skipstype}...")
    skipstype_dataframes = []
    for fil in skipstype_stier:
        gdf = gpd.read_parquet(fil)
        skipstype_dataframes.append(gdf)
    
    skipstype_data = pd.concat(skipstype_dataframes)
    print(f"Hentet {len(skipstype_data)} rader for skipstype {ønsket_skipstype}")
    display(skipstype_data.head())
else:
    print(f"Ingen data funnet for skipstype {ønsket_skipstype}")

Ingen mappe funnet for 2025-01
Ingen data funnet for skipstype 30


In [35]:
# 1. Opprett en mappe for partisjonerte data
output_mappe = "Data/partitioned_minutes"
os.makedirs(output_mappe, exist_ok=True)

# 2. Lag en kopi for å unngå SettingWithCopyWarning
data_for_partisjonering = within_area.copy()

# 3. Bruk .loc for å legge til 10-minutters intervall kolonne
# Dette gir intervaller 0-143 for hele dagen (144 intervaller på 10 minutter)
data_for_partisjonering.loc[:, 'minuttgruppe'] = (
    data_for_partisjonering['date_time_utc'].dt.hour * 6 + 
    data_for_partisjonering['date_time_utc'].dt.minute // 10
)

# 4. Partisjonering basert på minuttgruppe og ship_type
for minuttgruppe, gruppe_minutt in data_for_partisjonering.groupby('minuttgruppe'):
    # Konverter minuttgruppe til time og minutt for mappe-strukturen
    time = minuttgruppe // 6
    minutt = (minuttgruppe % 6) * 10
    
    # Lag en lesbar mappestruktur (time_minutt=HH_MM)
    minutt_mappe = os.path.join(output_mappe, f"time_minutt={time:02d}_{minutt:02d}")
    os.makedirs(minutt_mappe, exist_ok=True)
    
    for ship_type, gruppe_final in gruppe_minutt.groupby('ship_type'):
        skip_mappe = os.path.join(minutt_mappe, f"ship_type={ship_type}")
        os.makedirs(skip_mappe, exist_ok=True)
        
        # Bruk en kopi av dataene
        data_å_lagre = gruppe_final.copy()
        
        # Fjern partisjoneringskolonnen før lagring
        fil_sti = os.path.join(skip_mappe, f"data.parquet")
        data_å_lagre.drop('minuttgruppe', axis=1).to_parquet(fil_sti)
        
        print(f"Skrevet {len(gruppe_final)} rader til {fil_sti}")

### Strømming


Simulerer datastrømming over timer...

Datastrømming komplett!


In [32]:
# Simulert datastrømming over 10-minutters intervaller
partisjon_mappe = "Data/partitioned_minutes"

# Finn alle minutt-mapper sortert kronologisk
alle_intervaller = sorted([d for d in os.listdir(partisjon_mappe) if d.startswith("time_minutt=")])

print("\nSimulerer datastrømming over 10-minutters intervaller...")
for intervall_dir in alle_intervaller:
    # Hent time og minutt fra mappenavnet
    match = re.search(r"time_minutt=(\d{2})_(\d{2})", intervall_dir)
    if match:
        time, minutt = match.groups()
        
        intervall_sti = os.path.join(partisjon_mappe, intervall_dir)
        print(f"\nProsesserer data for {time}:{minutt}...")
        
        # Finn alle geoparquet-filer for dette intervallet
        filer = glob.glob(os.path.join(intervall_sti, "**/*.parquet"), recursive=True)
        
        # Les og bearbeid hver fil
        intervall_data = []
        for fil in filer:
            gdf = gpd.read_parquet(fil)
            intervall_data.append(gdf)
            
        if intervall_data:
            samlet_data = pd.concat(intervall_data)
            print(f"Kl. {time}:{minutt}: Lastet {len(samlet_data)} rader fra {len(filer)} filer")
            
            # Her kan du gjøre din analyse for dette intervallet
            print(f"  Gjennomsnittlig hastighet (SOG): {samlet_data['speed_over_ground'].mean():.2f}")
            print(f"  Antall unike skip: {samlet_data['mmsi'].nunique()}")
            
            # Sorter data etter nøyaktig tidspunkt innen intervallet
            samlet_data_sortert = samlet_data.sort_values('date_time_utc')
            
            # Her kunne du lagt til mer analyse eller visualisering
            # For eksempel se på hvordan skip beveger seg innen dette 10-minutters intervallet
            
        else:
            print(f"Ingen data funnet for kl. {time}:{minutt}")
        
        # Simuler tid mellom intervaller (kortere siden dette er en mer detaljert simulering)
        time_module.sleep(0.5)  # Vent et halvt sekund mellom hvert intervall
        
print("\nDatastrømming komplett!")


Simulerer datastrømming over 10-minutters intervaller...

Datastrømming komplett!


## Main

In [33]:
# ## Konvertere én enkelt fil - eksempel

# Definerer stier basert på den opprettede mappestrukturen
if os.path.exists(ROT_FOLDER):
    rå_mappe = os.path.join(ROT_FOLDER, "raw")
    prosessert_mappe = os.path.join(ROT_FOLDER, "processed")

    # Eksempelfil - dette er for demonstrasjon,
    eksempelfil = os.path.join(rå_mappe, "veier_kristiansand.geojson")

    # Utfør kun hvis filen eksisterer (endre denne kommentaren hvis du har en faktisk fil)
    if os.path.exists(eksempelfil):
        print(f"Konverterer {eksempelfil}...")
        resultat = converter_parquet(eksempelfil)
        if resultat:
            print(f"Filen ble konvertert og lagret til: {resultat}")

            # Lese og vise data
            try:
                gdf = gpd.read_parquet(resultat)
                display(gdf.head())

                # Enkel kartvisning
                import matplotlib.pyplot as plt
                fig, ax = plt.subplots(figsize=(10,8))
                gdf.plot(ax=ax)
                plt.title("Konvertert GeoParquet-fil")
                plt.show()
            except Exception as e:
                print(f"Kunne ikke vise data: {e}")
    else:
        print(f"Ingen eksempelfil funnet på {eksempelfil}")
        print("For å teste konvertering:")
        print(f"1. Legg en geografisk fil (f.eks. GeoJSON eller Shapefile) i {rå_mappe}")
        print(f"2. Kjør denne cellen på nytt med korrekt filsti")
else:
    print(f"Rotmappe {ROT_FOLDER} finnes ikke. Kjør seksjon 8 først.")

Ingen eksempelfil funnet på Data\raw\veier_kristiansand.geojson
For å teste konvertering:
1. Legg en geografisk fil (f.eks. GeoJSON eller Shapefile) i Data\raw
2. Kjør denne cellen på nytt med korrekt filsti


In [ ]:
import pandas as pd


geo_parquet_file = "data/hais_2024-01-01.snappy.parquet"

# Les filen
df = pd.read_parquet(geo_parquet_file)

# Filtrer kun LOFOTEN og SUPERSPEED 1
df = df[df["ship_name"].isin(["LOFOTEN", "SUPERSPEED 1"])]
print("Etter skipfilter:", len(df))


# Konvertere dato/tid
df["date_time_utc"] = pd.to_datetime(df["date_time_utc"])
df["hour"] = df["date_time_utc"].dt.hour

# Sorter på tid 
df = df.sort_values(by="date_time_utc").reset_index(drop=True)

df["ship_name"] = df["ship_name"].astype(str)  # eller .astype("string")

df.to_parquet(
    "ais_data_partitioned",
    partition_cols=["ship_name", "hour"],
    index=False,
)


import pyarrow.parquet as pq

filters = [
    ("ship_name", "=", "SUPERSPEED 1"),  # valgfritt
    ("hour", ">=", 12),
    ("hour", "<=", 14)
]

table = pq.read_table(
    "ais_data_partitioned",
    filters=filters,
    columns=["date_time_utc", "ship_name", "longitude", "latitude", "geometry"]
)
df_temp = table.to_pandas()

print("Antall rader kl. 12–14:", len(df_temp))


import folium

# Sorter dataene kronologisk
df_sorted = df_temp.sort_values(by="date_time_utc")

# Hent første og siste rad
df_first = df_sorted.head(1)
df_last = df_sorted.tail(1)

# Finn midtpunkt for kartet
center_lat = df_sorted["latitude"].mean()
center_lon = df_sorted["longitude"].mean()


# Visualisering avstand
# Lag Folium-kart
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Plot første punkt (grønn markør)
for _, row in df_first.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"Første posisjon: {row['ship_name']} - {row['date_time_utc']}",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(m)

# Plot siste punkt (rød markør)
for _, row in df_last.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"Siste posisjon: {row['ship_name']} - {row['date_time_utc']}",
        icon=folium.Icon(color="red", icon="stop")
    ).add_to(m)

# Tegn linje mellom første og siste punkt
# (Hent ut lat/lon fra df_first og df_last)
start_coords = [df_first["latitude"].iloc[0], df_first["longitude"].iloc[0]]
end_coords = [df_last["latitude"].iloc[0], df_last["longitude"].iloc[0]]
folium.PolyLine(locations=[start_coords, end_coords],
                color="blue",
                weight=2,
                opacity=0.7).add_to(m)

# Vis kartet
m

